## For CBT.01 only -- add timing files

In [1]:
import os
import pandas as pd
from scipy.io import loadmat
import numpy as np

In [2]:
data_path = "/Users/lj146/Documents/CBT/Data/behavioral/"

### filter participants

In [3]:
participants = []
for d in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, d)) and d.startswith("19") and len(d) == 5:
        participants.append(d)

In [4]:
def get_feedback_times(filename):
    f = loadmat(filename)
    trigger_times = f['trigger_feedback_times'].reshape(-1)
    strategy_times = f['strategy_feedback_times'].reshape(-1)
    return strategy_times[~np.isnan(strategy_times)], trigger_times[~np.isnan(trigger_times)]

In [5]:
def write_to_file(filename, times, duration=5, delimiter="\t"):
    with open(filename, "w") as f:
        for t in times:
            f.write(delimiter.join([str(t), str(duration), "1"]) + "\n")

In [6]:
# picked one
for par in participants:    
    print("###: {}".format(par))
    all_files = os.listdir(os.path.join(data_path, par))
    for file in all_files:
        # useful file
        if file.startswith("CBT") and "run" in file and "fixation" not in file:
            # get run number
            run = file.split("_")[3].rstrip(".mat")
            s, t = get_feedback_times(os.path.join(data_path, par, file))
            write_to_file(os.path.join(data_path, par, "timing_files", "standard", "CBT_train" + run + "_STRATEGY_FEEDBACK.txt"), s)
            write_to_file(os.path.join(data_path, par, "timing_files", "standard", "CBT_train" + run + "_TRIGGER_FEEDBACK.txt"), t)

###: 19239
###: 19338
###: 19357
###: 19418
###: 19425
###: 19480
###: 19492
###: 19497
###: 19589
###: 19636
###: 19658
###: 19671
###: 19761


### because localizers' files didn't record feedback times...

In [7]:
def create_feedback_file(filename, filepath, delta=30, duration=5):
    df = pd.read_csv(os.path.join(filepath, filename), delimiter='\t', header=None)
    df[0] = df[0].apply(lambda x : x + delta)
    assert len(df[1].unique()) == 1
    value = df[1].unique()[0]
    df[1] = df[1].map({value : duration})
    new_file_name = filename[:-4] + "_FEEDBACK.txt"
    df.to_csv(os.path.join(filepath, new_file_name), sep='\t', header=None, index=None)

In [8]:
for par in participants:    
    timing_file_path = os.path.join(data_path, par, "timing_files", "standard")
    for f in os.listdir(timing_file_path):
        if "localizer" in f and ("TRIGGER" in f or "STRATEGY" in f):
            create_feedback_file(f, timing_file_path)